In [2]:
import os
import torch
from transformers import AutoModelForCausalLM
from peft import PeftModel
import safetensors.torch
from transformers import AutoTokenizer
os.chdir("/app")

# Original tokenizer path
original_model_path = "Qwen/Qwen2.5-Coder-3B-Instruct"
# Rutas de entrada y salida
base_model_path = "Qwen2.5-Coder-3B-Instruct-JavaSpring-trained"
output_model_path = "Qwen2.5-Coder-3B-Instruct-JavaSpring-trained-merged"

# Crear directorio de salida si no existe
os.makedirs(output_model_path, exist_ok=True)

# Cargar el modelo base
model = AutoModelForCausalLM.from_pretrained(base_model_path)

# Cargar y fusionar el adaptador LoRA
adapter_folder = base_model_path  # La carpeta del adaptador
model = PeftModel.from_pretrained(model, adapter_folder)
model = model.merge_and_unload()  # Fusionar LoRA con el modelo base

# Resolver la compartición de tensores
state_dict = model.state_dict()

# Clonar tensores compartidos
if "model.embed_tokens.weight" in state_dict and "lm_head.weight" in state_dict:
    if state_dict["model.embed_tokens.weight"].data_ptr() == state_dict["lm_head.weight"].data_ptr():
        state_dict["lm_head.weight"] = state_dict["lm_head.weight"].clone()

# Guardar el modelo completo en formato safetensors con metadata
output_safetensors_path = os.path.join(output_model_path, "model.safetensors")
metadata = {"format": "pt"}  # Add metadata indicating PyTorch format
safetensors.torch.save_file(state_dict, output_safetensors_path, metadata=metadata)
print(f"Modelo fusionado guardado con metadata en: {output_safetensors_path}")

# Copiar otros archivos del modelo original a la nueva carpeta
files_to_copy = [
    "tokenizer_config.json",
    "tokenizer.json",
    "vocab.json",
    "special_tokens_map.json",
    "added_tokens.json",
    "merges.txt",
    "README.md", 
    "config.json",
    "generation_config.json",
]

for file in files_to_copy:
    src = os.path.join(base_model_path, file)
    dst = os.path.join(output_model_path, file)
    if os.path.exists(src):
        with open(src, "rb") as fsrc, open(dst, "wb") as fdst:
            fdst.write(fsrc.read())
            print(f"Archivo copiado: {file}")

print(f"Todos los archivos necesarios han sido copiados a: {output_model_path}")



# Reload and save tokenizer and config.json from the original model path
try:
    # Load the tokenizer from the original model path
    tokenizer = AutoTokenizer.from_pretrained(original_model_path)
    tokenizer.save_pretrained(output_model_path)
    print(f"Tokenizer re-saved to: {output_model_path}")
    
    # Load and save the config.json from the original model path
    config_src = os.path.join(original_model_path, "config.json")
    config_dst = os.path.join(output_model_path, "config.json")
    if os.path.exists(config_src):
        with open(config_src, "rb") as fsrc, open(config_dst, "wb") as fdst:
            fdst.write(fsrc.read())
            print(f"Archivo copiado: config.json")
    else:
        print(f"Advertencia: config.json no existe en el modelo original y no se pudo copiar. Descargar a mano desde el modelo original {original_model_path}.")
except Exception as e:
    print(f"Error al recargar o guardar el tokenizer y config.json: {str(e)}")

print(f"Tokenizer re-saved to: {output_model_path}")

Loading checkpoint shards: 100%|██████████| 2/2 [00:01<00:00,  1.07it/s]


Modelo fusionado guardado con metadata en: Qwen2.5-Coder-3B-Instruct-JavaSpring-trained-merged/model.safetensors
Archivo copiado: tokenizer_config.json
Archivo copiado: tokenizer.json
Archivo copiado: vocab.json
Archivo copiado: special_tokens_map.json
Archivo copiado: added_tokens.json
Archivo copiado: merges.txt
Archivo copiado: README.md
Archivo copiado: config.json
Archivo copiado: generation_config.json
Todos los archivos necesarios han sido copiados a: Qwen2.5-Coder-3B-Instruct-JavaSpring-trained-merged
Tokenizer re-saved to: Qwen2.5-Coder-3B-Instruct-JavaSpring-trained-merged
Advertencia: config.json no existe en el modelo original y no se pudo copiar. Descargar a mano desde el modelo original Qwen/Qwen2.5-Coder-3B-Instruct.
Tokenizer re-saved to: Qwen2.5-Coder-3B-Instruct-JavaSpring-trained-merged
